In [7]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import re

/speech/nency/miniconda3/envs/BERTTopic/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-01 15:45:13.814138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-01 15:45:13.814310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-09-01 15:45:13.814331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentio

In [37]:
#This cell contains code to augment data using NLP augmentation libraries

def augment_title(line):
    augmented_title = []
    splitted_text = line.split("|")
    pmid = splitted_text[0]
    aug = naw.SynonymAug()
    aug_title = aug.augment(splitted_text[2], n=1)
    new_title = pmid + "|t|" + aug_title[0]
    augmented_title.append(new_title)
    
#     aug = naw.SpellingAug()
#     aug_title = aug.augment(splitted_text[2], n=1)
#     new_title = pmid + "|t|" + aug_title[0]
#     augmented_title.append(new_title)

    aug = naw.ContextualWordEmbsAug()
    aug_title = aug.augment(splitted_text[2], n=1)
    new_title = pmid + "|t|" + aug_title[0]
    augmented_title.append(new_title)
    return augmented_title

def augment_abstract(line):
    augmented_abstract = []
    splitted_text = line.split("|")
    pmid = splitted_text[0]
    aug = naw.SynonymAug()
    aug_abstract = aug.augment(splitted_text[2], n=1)
    new_abstract = pmid + "|a|" + aug_abstract[0]
    augmented_abstract.append(new_abstract)
    
#     aug = naw.SpellingAug()
#     aug_abstract = aug.augment(splitted_text[2], n=1)
#     new_abstract = pmid + "|a|" + aug_abstract[0]
#     augmented_abstract.append(new_abstract)

    aug = naw.ContextualWordEmbsAug()
    aug_abstract = aug.augment(splitted_text[2], n=1)
    new_abstract = pmid + "|a|" + aug_abstract[0]
    augmented_abstract.append(new_abstract)
    return augmented_abstract
    

In [41]:
# this code is collecting augmented title, augmented abstracts and all annotations in pmid_mesh nested dictionary. 
# pmid_mesh structure is {pmid:{mes_id}}
# mesh_id structure is {entity_name:[entity_name, meahid]}

#causal_abstracts_with_ner is created in bertgt_results_analysis

import re
with open("latest_cau_train_file.txt", "r", encoding="utf-8") as file:
    augmented_titles = []
    augmented_abstracts = []
    pmids = []
    mesh_id={}
    pmid_mesh = {}
    pmid = 0
    causal_relations = []
    for line in file:
        line = line.rstrip()
        p_title = re.compile('^([0-9]+)\|t\|(.*)$')
        p_abstract = re.compile('^([0-9]+)\|a\|(.*)$')
        p_annotation = re.compile('^([0-9]+)	([0-9]+)	([0-9]+)	([^\t]+)	([^\t]+)(.*)')
        if p_title.search(line):
            print(line)
            pmid_mesh[pmid] = mesh_id 
            mesh_id = {}
            augmented_titles.append(augment_title(line))
            m = p_title.match(line)
            pmid = m.group(1)
            
        elif p_abstract.search(line):
            m = p_abstract.match(line)
            pmid = m.group(1)
            augmented_abstracts.append(augment_abstract(line))

        elif p_annotation.search(line):  # annotation
            m = p_annotation.match(line)
            pmid = m.group(1)
            start = m.group(2)
            last = m.group(3)
            mention = m.group(4)
            type = m.group(5)
            id = m.group(6)
            mesh_id[mention]=[type,id]
        elif "GCD" in line:
            causal_relations.append(line)
        else:
            print(line)
        pmids.append(pmid)
    pmid_mesh[pmid] = mesh_id 


32172939|t|CFTR gene variants, epidemiology and molecular pathology.

30245141|t|The genetic landscape of Parkinson's disease.

15082523|t|Androgen receptor in prostate cancer.

28966576|t|Modifications and Trafficking of APP in the Pathogenesis of Alzheimer's Disease.

29173173|t|AR Copy Number and AR Signaling-directed Therapies in Castrationresistant Prostate Cancer.

28920889|t|Therapies targeting DNA and RNA in Huntington's disease.

34098113|t|Gene targeting techniques for Huntington's disease.

34934215|t|Cystic fibrosis: current concepts.

24631642|t|Genetics of cystic fibrosis: CFTR mutation classifications toward genotype-based CF therapies.

30798813|t|Hemochromatosis: Hereditary hemochromatosis and HFE gene.

34090606|t|Cystic fibrosis.

31904117|t|SCN1A-related phenotypes: Epilepsy and beyond.

31407125|t|Serum amyloid A1 is involved in amyloid plaque aggregation and memory decline in amyloid beta abundant condition.

33132897|t|Interleukin-17A: The Key Cytokine in Neurod

In [42]:
# this code is to write the annotated abstracts in proper pubtator format.
# we are getting mesh id (mention, type, id ) after running the above cell.
# augmented abstracts will be written in augmented_data.txt

number_of_pmids = len(set(pmids)) #number of unique pmids
with open('latest_train_augmented_data.txt','w', encoding='utf8') as file_Outputfile: 
    for j in range(number_of_pmids-1): # no of unique pmids
        for i in range(2): # because for each abstract we are doing 2 augmentations
            #j=0
            file_Outputfile.write("\n")
            file_Outputfile.write("\n")
            
            file_Outputfile.write(augmented_titles[j][i].replace("\n", "").lower()+"\n")
            
            file_Outputfile.write(augmented_abstracts[j][i].replace("\n", "").lower()+"\n")

            pmid = int(augmented_titles[j][i].split("|")[0])
            abstract = augmented_titles[j][i].split("|")[2]+" "+augmented_abstracts[j][i].split("|")[2]
            abstract=abstract.replace("\n", "").lower()
            print(abstract)
            mesh_id = pmid_mesh[str(pmid)]
            for mention, list1 in mesh_id.items():
                mention=mention.lower()
                type = list1[0]
                id = list1[1]
                indexes = [i for i in range(len(abstract) - len(mention) + 1) if abstract[i:i+len(mention)] == mention]
                print(mention, indexes)
                #indexes = [i for i in range(len(abstract)) if abstract.lower().startswith(mention.lower(), i)]

                if len(indexes)>0:
                    for ind in indexes:
                        start = ind
                        last = str(start+len(mention))
                        start=str(start)
                        file_Outputfile.write(str(pmid)+"\t"+start+"\t"+last+"\t"+mention.lower()+"\t"+type+id+"\n")
            for relation in causal_relations:

                if(pmid == int(relation.split("\t")[0])):
                    file_Outputfile.write(relation+"\n")



cftr gene variants, epidemiology and molecular pathology. pathogenic variants of the cftr gene are responsible for a across the board phenotypic spectrum characterized by malfunction of some exocrine tissues, with an autosomal recessive mode of inheritance. more than 2, 000 variants, distributed throughout the cftr gene, have been identified, with different effects on the gene and protein expression and function. genotype - phenotype coefficient of correlation studies have associated severe var. with a typical multi - organ form of cystic fibrosis, while mild variants are involved in monosymptomatic or grownup - onset diseases, called cftr - related disorders. however, the interpretation of rare variants remains challenging. this review presents an overview of the epidemiology of cftr variants worldwide and in france and describes the functional classification. finally, some frequent cystic fibrosis - causing and mild cftr variants are used as example to depict the molecular pathology 